In [ ]:
!pip install evaluate rouge_score accelerate -U # accelerate -U is needed for seq2seq training

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11

In [ ]:
from torch.utils.data import Dataset
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)
# from datasets import Dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5p-220m")
model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5p-220m")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/446M [00:00<?, ?B/s]

In [ ]:
texts = [
    {
        # https://github.com/pydantic/pydantic/pull/9111
        "code": """
Pull Request Diff:
```
diff --git a/pydantic/_internal/_fields.py b/pydantic/_internal/_fields.py
index 3f78a920d5..6e5e933061 100644
--- a/pydantic/_internal/_fields.py
+++ b/pydantic/_internal/_fields.py
@@ -177,7 +177,7 @@ def collect_model_fields(  # noqa: C901
             )

         # when building a generic model with `MyModel[int]`, the generic_origin check makes sure we don't get
-        # "... shadows an attribute" errors
+        # "... shadows an attribute" warnings
         generic_origin = getattr(cls, '__pydantic_generic_metadata__', {}).get('origin')
         for base in bases:
             dataclass_fields = {
#  Do not warn about shadowed fields if they are not redefined in a child class
```
""",
        "description": """
        # Change Summary

        Adds another early exit condition when evaluating whether to log a warning message during detection for shadowed fields.

In the case where a field is defined in a parent class, but it has not been defined at all in a child class, it is technically not a shadowed field, and so shouldn't be warned as such.

Note this is very different from the case where a child class does redefine a field but with a narrower type or even defined as the same type but with a different default. Conceptually this is probably ok, but checking for that is quite complex and this PR does not attempt to try. So this is about checking if a field is defined or not defined - if it is, regardless of type or default value, the warning message is still logged.
        """,
    },
    {
        # https://github.com/pydantic/pydantic/pull/9144
        "code": """
        --- a/pydantic/main.py
+++ b/pydantic/main.py
@@ -222,9 +222,12 @@ def model_construct(cls: type[Model], _fields_set: set[str] | None = None, **val
         fields_set = set()

         for name, field in cls.model_fields.items():
-            if field.alias and field.alias in values:
+            if field.alias is not None and field.alias in values:
                 fields_values[name] = values.pop(field.alias)
                 fields_set.add(name)
+            elif field.validation_alias is not None and field.validation_alias in values:
+                fields_values[name] = values.pop(field.validation_alias)
+                fields_set.add(name)
             elif name in values:
                 fields_values[name] = values.pop(name)
                 fields_set.add(name)
        """,
        "description": """
        # Change Summary

        just like you can construct a model using a field alias, this PR fixes constructing a model using validation_alias.
        """,
    },
    {
        "code": """
        diff --git a/pydantic/json_schema.py b/pydantic/json_schema.py
index 9f0ceb3e36..3e63ecc08d 100644
--- a/pydantic/json_schema.py
+++ b/pydantic/json_schema.py
@@ -751,6 +751,8 @@ def literal_schema(self, schema: core_schema.LiteralSchema) -> JsonSchemaValue:
             result['type'] = 'boolean'
         elif types == {list}:
             result['type'] = 'array'
+        elif types == {type(None)}:
+            result['type'] = 'null'
         return result

     def enum_schema(self, schema: core_schema.EnumSchema) -> JsonSchemaValue:
        """,
        "description": """
        # Change Summary

        This PR aims to complete #8944 and #8905 by also handling null types when generating a json-schema from a pydantic model.

For instance, the following model:
```python
from pydantic import BaseModel
from typing import Literal


class Foo(BaseModel):
    bar: Literal["Bar"] = 'Bar'
    baz: Literal[None] = None
    foo: str = 'Foo'
```
leads to:
```
{'properties': {'bar': {'const': 'Bar', 'default': 'Bar', 'enum': ['Bar'], 'title': 'Bar', 'type': 'string'}, 'baz': {'const': None, 'default': None, 'enum': [None], 'title': 'Baz', 'type': 'null'}, 'foo': {'default': 'Foo', 'title': 'Foo', 'type': 'string'}}, 'title': 'Foo', 'type': 'object'}
```
        """,
    },
]

In [ ]:
import evaluate

metrics_name = ["bleu", "rouge", "exact_match"]
for name in metrics_name:
    evaluate.load(name)
metrics = evaluate.combine(metrics_name)


def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    # Decode predictions and labels
    decoded_preds = [
        tokenizer.decode(p, skip_special_tokens=True, clean_up_tokenization_spaces=True) for p in predictions
    ]
    # Labels may have -100 for ignored indices, we need to filter these out
    decoded_labels = []
    for label in labels:
        # Replace -100 with pad_token_id to avoid decoding issues
        label = [token if token != -100 else tokenizer.pad_token_id for token in label]
        decoded_labels.append(tokenizer.decode(label, skip_special_tokens=True, clean_up_tokenization_spaces=True))

    # # Define metrics
    # rouge = evaluate.load('rouge')
    # bleu = evaluate.load('bleu')
    # exact_match = evaluate.load('exact_match')

    # # Compute metrics
    # rouge_score = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    # bleu_score = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    # exact_match_score = exact_match.compute(predictions=decoded_preds, references=decoded_labels)

    # # Extract metrics and format them
    # metrics = {
    #     "rouge1": rouge_score['rouge1'],
    #     "rouge2": rouge_score['rouge2'],
    #     "rougeL": rouge_score['rougeL'],
    #     "bleu": bleu_score['bleu'],
    #     "exact_match": exact_match_score['exact_match']
    # }

    return metrics.compute(predictions=decoded_preds, references=decoded_labels)

In [ ]:
class CodeDiffDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512, template="Code: {code} Description: "):
        """
        Initializes the dataset.
        :param data: List of dictionaries containing 'code' and 'description'.
        :param tokenizer: Tokenizer to be used for encoding texts.
        :param max_length: Maximum sequence length for inputs and targets.
        """
        self.dataset = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.template = template

    def __len__(self):
        """
        Returns the length of the dataset.
        """
        return len(self.dataset)

    def __getitem__(self, idx):
        """
        Retrieves an item by index.
        :param idx: Index of the item.
        :return: A dictionary containing input_ids, attention_mask, and labels.
        """
        instance = self.dataset[idx]
        code = instance["code"]
        description = instance["description"]

        # Tokenize the inputs and targets
        input_encodings = self.tokenizer(
            description, max_length=self.max_length, truncation=True, padding="max_length", return_tensors="pt"
        )
        target_encodings = self.tokenizer(
            code, max_length=self.max_length, truncation=True, padding="max_length", return_tensors="pt"
        )

        # Input_ids and attention_mask for the Seq2Seq model
        input_ids = input_encodings["input_ids"].squeeze()
        attention_mask = input_encodings["attention_mask"].squeeze()

        # Labels are the target token ids; we set padding tokens to -100 so they are not considered in loss computation
        labels = target_encodings["input_ids"].squeeze()
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

In [ ]:
train_dataset = CodeDiffDataset(texts, tokenizer)
eval_dataset = CodeDiffDataset(texts, tokenizer)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",  # output directory
    num_train_epochs=5,  # total number of training epochs
    # warmup_steps=500,                # number of warmup steps for learning rate scheduler
    # weight_decay=0.01,               # strength of weight decay
    # logging_dir='./logs',            # directory for storing logs
    logging_steps=1,
    evaluation_strategy="epoch",  # perform evaluation each epoch
    # eval_steps=2000,
    # # per_device_train_batch_size=8,
    # # per_device_eval_batch_size=64,
    # auto_find_batch_size=True,
    # save_strategy="epoch",
    # lr_scheduler_type=SchedulerType.COSINE_WITH_RESTARTS,
    predict_with_generate=True,
    fp16=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Precisions,Brevity Penalty,Length Ratio,Translation Length,Reference Length,Rouge1,Rouge2,Rougel,Rougelsum,Exact Match
1,0.997300,0.771141,0.000000,"[0.2564102564102564, 0.0, 0.0, 0.0]",0.000005,0.075435,39,517,0.029202,0.000000,0.023504,0.029202,0.000000
2,0.803800,0.634180,0.000000,"[1.0, 0.9565217391304348, 0.8571428571428571, 0.7894736842105263]",0.000000,0.048356,25,517,0.102375,0.077160,0.096314,0.102375,0.000000
3,0.745900,0.528420,0.000000,"[1.0, 0.8333333333333334, 0.7272727272727273, 0.65]",0.000000,0.050290,26,517,0.110303,0.070100,0.104242,0.110303,0.000000
4,0.707500,0.449967,0.000000,"[1.0, 0.8333333333333334, 0.7272727272727273, 0.65]",0.000000,0.050290,26,517,0.110303,0.070100,0.104242,0.110303,0.000000
5,0.650600,0.411195,0.000000,"[1.0, 0.8076923076923077, 0.7083333333333334, 0.5909090909090909]",0.000000,0.054159,28,517,0.116036,0.076159,0.104024,0.116036,0.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Trainer is attempting to log a value of "[0.2564102564102564, 0.0, 0.0, 0.0]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[1.0, 0.9565217391304348, 0.8571428571428571, 0.7894736842105263]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[1.0, 0.8333333333333334, 0.7272727272727273, 0.65]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is inco

TrainOutput(global_step=5, training_loss=0.7810169458389282, metrics={'train_runtime': 10.9167, 'train_samples_per_second': 1.374, 'train_steps_per_second': 0.458, 'total_flos': 9134368358400.0, 'train_loss': 0.7810169458389282, 'epoch': 5.0})